In [1]:
# GENERAL IMPORTS

import pandas as pd
import numpy as np
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

import datetime
import math
import bisect
import time
from os import listdir
from os.path import isfile, join

In [2]:
daydatadir = r'''C:\ProgramData\Kibot Agent\Data\SP500_day'''
files = [f for f in listdir(daydatadir) if isfile(join(daydatadir, f))]
instruments = [f[0:-4] for f in files]

colnames = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
daypricedata = {}
for f in instruments:
    df = pd.read_csv(join(daydatadir, f+'.txt'), names=colnames)
    df['Date'] = df['Date'].map(lambda x: datetime.date(int(x[6:10]), int(x[0:2]), int(x[3:5])))
    df['Gap'] = np.insert(np.array(df['Open'][1:])-np.array(df['Close'][:-1]),0,0)
    df = df.set_index('Date')
    daypricedata[f] = df

In [3]:
daypricedata['A']['Open'].iloc[0]

29.6

In [4]:
start_date = datetime.date(2018,1,1)
end_date = datetime.date(2018,7,1)
num_instruments = 1
max_gap_instruments = {}

for ii in range((end_date - start_date).days):
    date = start_date + datetime.timedelta(days=ii)
    max_gap = 0
    max_gap_instrument = None
    for instrument in instruments:
        if date in daypricedata[instrument].index:
            gap = abs(daypricedata[instrument].loc[date]['Gap'])/daypricedata[instrument].loc[date]['Open']
            if gap > max_gap:
                max_gap = gap
                max_gap_instrument = instrument
    max_gap_instruments[date] = max_gap_instrument
    print(date, max_gap_instrument, max_gap)
            
    

2018-01-01 None 0
2018-01-02 CPWR 0.08571428571428563
2018-01-03 SCG 0.18576729971621928
2018-01-04 LB 0.11999165275459098
2018-01-05 FOSL 0.0837628865979382
2018-01-06 None 0
2018-01-07 None 0
2018-01-08 STX 0.11964129483814522
2018-01-09 AYI 0.1325441609421002
2018-01-10 ISRG 0.060848622228518764
2018-01-11 XRX 0.0546287209682696
2018-01-12 GME 0.07887899034892344
2018-01-13 None 0
2018-01-14 None 0
2018-01-15 None 0
2018-01-16 CPWR 0.09833333333333324
2018-01-17 FAST 0.05017779533781113
2018-01-18 ALB 0.10202457784457614
2018-01-19 CPWR 0.03225806451612906
2018-01-20 None 0
2018-01-21 None 0
2018-01-22 FE 0.08468408865365526
2018-01-23 NFLX 0.10770437169182512
2018-01-24 GWW 0.11200859462040429
2018-01-25 NWL 0.11709337349397603
2018-01-26 INTC 0.06372023170993346
2018-01-27 None 0
2018-01-28 None 0
2018-01-29 DPS 0.24424329045577253
2018-01-30 ESRX 0.08560826928173879
2018-01-31 JNPR 0.09599999999999995
2018-02-01 CPWR 0.2585365853658536
2018-02-02 CPWR 0.16666666666666669
2018-02-

In [14]:
def gap_trading_strategy(portfolio, data_dict, param_dict):
    minutedatadir = r'''C:\ProgramData\Kibot Agent\Data\SP500_1'''
    minutecolnames = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    
    max_gap_instruments = data_dict['max_gap_instruments']
    
    for ii in range((end_date - start_date).days):
        date = start_date + datetime.timedelta(days=ii)
        
        instrument = max_gap_instruments[date]
        if instrument:
            try:
                pricedata = pd.read_csv(join(minutedatadir, instrument+'.txt'), names=minutecolnames)
                pricedata['DateTime'] = (pricedata['Date']+pricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
                pricedata = pricedata[[(dt >= datetime.datetime(date.year, date.month, date.day, 9, 30, 0)) and (dt <= datetime.datetime(date.year, date.month, date.day, 16, 0, 0)) for dt in pricedata['DateTime']]].copy()

                portfolio.positions[instrument] = 0
                print(date, instrument)
            except:
                continue
            
            if not pricedata.index.empty and len(pricedata) > 1:
                open_price = pricedata['Open'].iloc[1]
                for index in range(len(pricedata)):
                    row = pricedata.iloc[index]

                    if index == len(pricedata)-1:
                        if portfolio.positions[instrument] < 0:
                            portfolio.buy(instrument, -portfolio.positions[instrument], row)
                        elif portfolio.positions[instrument] > 0:
                            portfolio.sell(instrument, portfolio.positions[instrument], row)
                    elif row['Close'] > open_price:
                        if portfolio.positions[instrument] <= 0:
                            quantity = np.floor(10000/row['Close'])
                            portfolio.buy(instrument, -portfolio.positions[instrument] + quantity, row)
                    elif row['Close'] < open_price:
                        if portfolio.positions[instrument] >= 0:
                            quantity = np.floor(10000/row['Close'])
                            portfolio.sell(instrument, portfolio.positions[instrument] + quantity, row)

                    portfolio.update(instrument, row)
    
    portfolio.portfolio_history = pd.DataFrame(portfolio.portfolio_history)
    portfolio.transaction_history = pd.DataFrame(portfolio.transaction_history)                

In [15]:
class Portfolio:
    def __init__(self, starting_cash=10000, commission=0):
        self.cash = starting_cash
        self.positions = {}
        self.commission = commission

        self.ask_price_key = 'Close'
        self.bid_price_key = 'Close'
        
        self.net_value = 0
        self.portfolio_history = []
        self.transaction_history = []
        
    def update(self, instrument, row):
        price = row[self.bid_price_key]
        self.net_value = self.cash + self.positions[instrument] * price
        self.portfolio_history.append(
            {
                "DateTime": row['DateTime'],
                "Cash": self.cash,
                "Position": self.positions,
                "Net Value": self.net_value
            }
        )
        
    def buy(self, instrument, quantity, row):
        price = row[self.ask_price_key]
        cost = quantity*price + self.commission
        self.cash -= cost

        if not instrument in self.positions.keys():
            positions[instrument] = 0
        self.positions[instrument] += quantity

        self.cash = round(self.cash*100)/100

        self.transaction_history.append(
            {
                "DateTime": row['DateTime'],
                "Instrument" : instrument,
                "Price": price,
                "Quantity": quantity
            }
        )
        
    def sell(self, instrument, quantity, row):
        price = row[self.bid_price_key]
        self.cash += quantity*price - self.commission

        if not instrument in self.positions.keys():
            positions[instrument] = 0
        self.positions[instrument] -= quantity

        self.cash = round(self.cash*100)/100

        self.transaction_history.append(
            {
                "DateTime": row['DateTime'],
                "Instrument" : instrument,
                "Price": price,
                "Quantity": -quantity
            }
        )

In [16]:
data_dict = {'max_gap_instruments': max_gap_instruments}
portfolio = Portfolio(starting_cash = 10000, commission=5)

param_dict = {}
gap_trading_strategy(portfolio, data_dict, param_dict)

2018-01-03 SCG
2018-01-04 LB
2018-01-05 FOSL
2018-01-08 STX
2018-01-09 AYI
2018-01-10 ISRG
2018-01-11 XRX
2018-01-12 GME
2018-01-17 FAST
2018-01-18 ALB
2018-01-22 FE
2018-01-23 NFLX
2018-01-24 GWW
2018-01-25 NWL
2018-01-26 INTC
2018-01-29 DPS
2018-01-30 ESRX
2018-01-31 JNPR
2018-02-07 CMG
2018-02-08 COTY
2018-02-09 EXPE
2018-02-12 CSRA
2018-02-13 ABC
2018-02-14 FOSL
2018-02-15 TRIP
2018-02-16 VFC
2018-02-20 WMT
2018-02-21 DVN
2018-02-22 SRCL
2018-02-26 DF
2018-02-27 MNK
2018-02-28 FTR
2018-03-01 PDCO
2018-03-02 JCP
2018-03-05 XL
2018-03-06 CIEN
2018-03-07 DLTR
2018-03-08 ESRX
2018-03-09 BIG
2018-03-12 MU
2018-03-14 SIG
2018-03-15 ALXN
2018-03-16 JBL
2018-03-20 ORCL
2018-03-22 ABBV
2018-03-23 KBH
2018-03-27 RHT
2018-03-28 CXO
2018-03-29 GME
2018-04-04 AYI
2018-04-05 FB
2018-04-06 MU
2018-04-09 LUK
2018-04-10 STX
2018-04-11 FAST
2018-04-12 BBBY
2018-04-13 AVGO
2018-04-16 JBHT
2018-04-17 NFLX
2018-04-18 TXT
2018-04-19 PM
2018-04-20 GE
2018-04-23 AA
2018-04-24 WAT
2018-04-25 TER
2018-04-26

In [8]:
class ProfitCharts():
    def __init__(self, portfolio):
        self.portfolio = portfolio
        cash = portfolio.portfolio_history['Cash']
        net_value = portfolio.portfolio_history['Net Value']
        
        x_vals = [dt.strftime("%y/%m/%d, %H:%M:%S") for dt in portfolio.portfolio_history['DateTime']]
        tick_vals = [ii for ii in range(len(x_vals)) if 
                                 portfolio.portfolio_history['DateTime'][ii].time() == datetime.time(9, 30)]
        xaxisdatetimes = [portfolio.portfolio_history['DateTime'][ii] for ii in tick_vals]
        tick_text = [dt.strftime(' %y/%m/%d ') if dt.time() == datetime.time(9, 30)
                                 else dt.strftime(' %H:%M ') for dt in xaxisdatetimes]

        data1 = [ dict(
                type = 'scatter',
                x = x_vals,
                y = cash,
                name = 'Cash'),

                dict(
                type = 'scatter',
                x = x_vals,
                y = net_value - cash,
                name = 'Position'),

                dict(
                type = 'scatter',
                x = x_vals,
                y = net_value,
                name = 'Net Value')
               ]
        
        layout1 = dict(
                title = 'Profit Chart',
                xaxis = dict(
                    title = dict(
                        text = 'DateTime'
                    ),
                    type = 'category', 
                    categoryorder = 'category ascending',
                    tickvals = tick_vals,
                    ticktext = tick_text
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Value'
                    )
                ))
        
        self.chart1 = go.FigureWidget( data=data1, layout=layout1)
        
        data2 = [
                dict(
                type = 'scatter',
                x = x_vals,
                y = [nv - net_value[0] for nv in net_value],
                name = 'Profit')
               ]
        
        layout2 = dict(
                title = 'Profit Chart',
                xaxis = dict(
                    title = dict(
                        text = 'DateTime'
                    ),
                    type = 'category', 
                    categoryorder = 'category ascending',
                    tickvals = tick_vals,
                    ticktext = tick_text
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Profit'
                    )
                ))
        
        self.chart2 = go.FigureWidget( data=data2, layout=layout2)        
        # DEFINE DISPLAY CONFIGURATION
        self.display = ipywidgets.VBox([self.chart1, self.chart2])

In [10]:
portfolio.transaction_history

,DateTime,Instrument,Price,Quantity
0,2018-01-03 09:30:00,SCG,46.0400,217.0
1,2018-01-03 09:39:00,SCG,45.6300,-436.0
2,2018-01-03 09:40:00,SCG,45.8700,437.0
3,2018-01-03 09:42:00,SCG,45.6900,-436.0
4,2018-01-03 10:03:00,SCG,45.9300,435.0
5,2018-01-03 10:04:00,SCG,45.7700,-435.0
6,2018-01-03 10:05:00,SCG,45.8300,436.0
7,2018-01-03 10:07:00,SCG,45.6900,-436.0
8,2018-01-03 10:08:00,SCG,45.8200,436.0
9,2018-01-03 10:09:00,SCG,45.6300,-437.0


In [17]:
profit_charts = ProfitCharts(portfolio)
profit_charts.display

    'data': [{'name': 'Cash',
              'type': 'scatter',
              'ui…